In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.python.client import device_lib
print device_lib.list_local_devices()

In [ ]:
import tensorflow as tf
import numpy as np
from StochasticDropoutNet_unrolled import StochasticDropoutNet
from util import xor_data, extract_update_dict
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=False)

In [ ]:
# model = StochasticDropoutNet(min_dropout_rate = 0.001, max_dropout_rate = 0.001, num_weight_train_steps = 64)
model = StochasticDropoutNet(min_init_dropout_rate = 0.001, 
                             max_init_dropout_rate = 0.001,
                             train_batch_size = 128,
                             valid_batch_size = 256)

In [ ]:
inputs_train, targets_train = mnist.train.next_batch(55000)
inputs_train = inputs_train.reshape((-1, 28, 28, 1))
inputs_valid, targets_valid = mnist.validation.next_batch(5000)
inputs_valid = inputs_valid.reshape((-1, 28, 28, 1))

model.train_unroll(inputs_train, targets_train, inputs_valid, targets_valid, num_epochs = 40)
model.train_fine_tune(inputs_train, targets_train, inputs_valid, targets_valid, num_epochs = 20)

In [ ]:
inputs_test, targets_test = mnist.test.next_batch(10000)
inputs_test = inputs_test.reshape((-1, 28, 28, 1))
accs = []
for i in range(10):
    feed_dict = {model.inputs: inputs_test[i*1000:(i+1)*1000, ...], 
                 model.targets: targets_test[i*1000:(i+1)*1000, ...]}
    acc = model.sess.run(model.accuracy, feed_dict=feed_dict)
    accs.append(acc)
    
print(sum(accs)/10)

In [ ]:
import csv
with open('./log/results.csv', 'a+') as f:
    writer = csv.writer(f)
    writer.writerow(model.train_losses)
    writer.writerow(model.valid_losses)

In [ ]:
feed_dict = {model.inputs: inputs_train[0 : 32, ...],
             model.targets: targets_train[0 : 32, ...]}
print(model.pred)
print(model.sess.run(model.pred, feed_dict = feed_dict))
print(model.sess.run(model.targets, feed_dict = feed_dict))
print(model.struct_train_op)

In [ ]:
inputs_train, targets_train = mnist.train.next_batch(55000)
a = (targets_train.shape)
print(a)